In [117]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests as rq
import json
import time
import random

# Login to ResearchGate

In [118]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.window import WindowTypes

driver = webdriver.Firefox()
driver.get("https://www.researchgate.net/login")

password = driver.find_element(By.ID, "input-password")
username = driver.find_element(By.ID, "input-login")

password.send_keys("kcaeDHT8uE5wYMe")
username.send_keys("matanki@my.hit.ac.il")
password.send_keys(Keys.RETURN)

In [119]:
fullname = []
gender = []
position = []
nation = []
institute = []
shanghai_ranking = []
depratment = []
disciplines = []
publications = []
citations = []
recommendations = []
top_cited_research = []
# top_cited_research_journal = []
interest_score = []
reads = []
h_index = []
URL = []

## List of institues & Shanghai ranking dict.

In [120]:
instLst = ["Holon-Institute-of-Technology", "Tel Aviv University", "Hebrew University of Jerusalem", "Bar Ilan University", "Technion-Israel Institute of Technology", "The-Open-University-of-Israel", "Ben-Gurion-University-of-the-Negev", "Ariel-University", "University-of-Haifa", "Weizmann-Institute-of-Science", "Academic College of Tel Aviv-Yafo", "Shamoon-College-of-Engineering"]

# dictionary
inst_rank = {
    "Holon-Institute-of-Technology":801,
    "Tel Aviv University":151,
    "Hebrew University of Jerusalem":77,
    "Bar Ilan University":301,
    "Technion-Israel Institute of Technology":83,
    "The-Open-University-of-Israel":201,
    "Ben-Gurion-University-of-the-Negev":401,
    "Ariel-University":801,
    "University-of-Haifa":501,
    "Weizmann-Institute-of-Science":83,
    "Academic College of Tel Aviv-Yafo":801,
    "Shamoon-College-of-Engineering":801
}

inst_nation = {
    "Holon-Institute-of-Technology":"Israel",
    "Tel Aviv University":"Israel",
    "Hebrew University of Jerusalem":"Israel",
    "Bar Ilan University":"Israel",
    "Technion-Israel Institute of Technology":"Israel",
    "The-Open-University-of-Israel":"Israel",
    "Ben-Gurion-University-of-the-Negev":"Israel",
    "Ariel-University":"Israel",
    "University-of-Haifa":"Israel",
    "Weizmann-Institute-of-Science":"Israel",
    "Academic College of Tel Aviv-Yafo":"Israel",
    "Shamoon-College-of-Engineering":"Israel"
}

## Determine the gender by name using API

In [121]:
def genderFromName(string):
    name = string.split()
    name = name[0]
#     print(name) # taking only firstname
    url = "https://api.genderize.io?name="+name
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:102.0) Gecko/20100101 Firefox/102.0'}
    file = rq.get(url, headers=headers)
    if str(file.content).find("female") == -1: #TODO: maybe a better way in tame consuming.
        return "male"
    else:
        return "female"

## Crawling

In [122]:
for x in instLst:
    driver.get("https://www.researchgate.net/institution/"+x+"/members")
    content = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/div[3]/div/div/nav/div[7]/a/span')
    index = bs(content.get_property("innerHTML"), "html.parser").get_text()
#     print(index) # Number of member pages for each institue
    
    limit = min(int(index) + 1, 101) # we don't mind more than 100*10 researchers per institue
    
    for i in range(1, limit): #int(index) + 1
        
        driver.get("https://www.researchgate.net/institution/"+x+"/members/"+str(i))
        p = driver.current_window_handle
        
        for j in range(1, 11):

            try:
                name = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/div[2]/div/div["+str(j)+"]/div/div/div[2]/div/div[1]/div/div/h5/a")
                url = name.get_attribute('href')
                name = bs(name.get_property("innerHTML"), "html.parser").get_text()
                print(url)
                print(name)
            except:
                break #no more people to show
            
            Gender = genderFromName(name)
            Gender = "Nan"
            print (Gender)
            print(x)
            print(inst_rank[x])
            try:
                Department = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/div[2]/div/div["+str(j)+"]/div/div/div[2]/div/div[2]/div/div[2]/ul/li/span")
                Department = bs(Department.get_property("innerHTML"), "html.parser").get_text()
                print(Department)
            except:
                Department = "Nan"
                print("Nan")

            try: 
                Disciplines = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div[2]/div/div[1]/div/div[2]/div/div["+str(j)+"]/div/div/div[2]/div/div[3]/div/div[2]")
                Disciplines = bs(Disciplines.get_property("innerHTML"), "html.parser").get_text("|", strip=True)
#                 A = [text for text in disciplines.stripped_strings] # getting list of disciplines instead of on string
                print(Disciplines)
            except:
                Disciplines = "Nan"
                print("Nan")
            
            # all data on the members' page was collected    
            # now we want to reach the web page of the sientist

            driver.switch_to.new_window(WindowTypes.TAB)
            driver.get(url+'/stats')
            
            try: 
                Research_Interest_Score = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[1]/div/div/div/div[2]/ul/li[1]/div/div/div/div[1]/div")
                Research_Interest_Score = bs(Research_Interest_Score.get_property("innerHTML"), "html.parser").get_text()
                if Research_Interest_Score == '- -':
                    Research_Interest_Score = 0;
                print(Research_Interest_Score)
            except:
                Research_Interest_Score = "0"
                print("Nan")

            try: 
                Reads = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[1]/div/div/div/div[2]/ul/li[2]/div/div/div/div[1]/div")
                Reads = bs(Reads.get_property("innerHTML"), "html.parser").get_text()
                print(Reads)
            except:
                Reads = "0"
                print("Nan")

            try: 
                Citations = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[1]/div/div/div/div[2]/ul/li[3]/div/div/div/div[1]/div")
                Citations = bs(Citations.get_property("innerHTML"), "html.parser").get_text()
                print(Citations)
            except:
                Citations = "0"
                print("Nan")

            try: 
                Recommendations = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[1]/div/div/div/div[2]/ul/li[4]/div/div/div/div[1]/div")
                Recommendations = bs(Recommendations.get_property("innerHTML"), "html.parser").get_text()
                print(Recommendations)
            except:
                Recommendations = "0"
                print("Nan")

            try: 
                hindex = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[3]/div/div/div/div[2]/ul/li[1]/ul/li[1]/div/div/div/div[1]/div")
                hindex = bs(hindex.get_property("innerHTML"), "html.parser").get_text()
                print(hindex)
            except:
                hindex = "0"
                print("Nan")

            try: 
                Top_cited_research = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[2]/div[1]/div/div/div[3]/div/div/div/div[2]/ul/li[2]/div/div/div/div[2]/div/a/strong")
                Top_cited_research = bs(Top_cited_research.get_property("innerHTML"), "html.parser").get_text()
                print(Top_cited_research)
            except:
                Top_cited_research = "Nan"
                print("Nan")

            try: 
                Position = driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/div[1]/div/div/div/div[1]/div[1]/div[1]/div/div/div[2]/div/div[3]/ul/li[1]")
                Position = bs(Position.get_property("innerHTML"), "html.parser").get_text()
                word = Position.split()[0]
                if word == x.split()[0] or word == x.split('-')[0]:
                    Position = "Nan"
                print(Position)
            except:
                Position = "Nan"
                print("Nan")
            
            try: 
                no_publications = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[1]/div/div/div/div[1]/div[2]/div/div/div/div[1]/div/nav/div/div[1]/a[2]/div/div')
                no_publications = bs(no_publications.get_property("innerHTML"), "html.parser").get_text()

                # reciving the number of publications

                no_publications = no_publications.split()[1].strip('()')
                print(no_publications)
                
            except:
                no_publications = "0"
                print("0")
            

            # now we've done scraping all relevent data on this sientinst
            # we close the tab and return to previous tab
            
            time.sleep(random.uniform(1.9, 2.4))
            driver.close()
            driver.switch_to.window(p)
                
            print("\n")

            fullname.append(name)
            gender.append(Gender)
            position.append(Position)
            nation.append(inst_nation[x])
            institute.append(x)
            shanghai_ranking.append(inst_rank[x])
            depratment.append(Department)
            disciplines.append(Disciplines)
            publications.append(float(no_publications.replace(",", "").replace(".", "")))
            citations.append(float(Citations.replace(",", "").replace(".", "")))
            recommendations.append(float(Recommendations.replace(",", "").replace(".", "")))
            top_cited_research.append(Top_cited_research)
            # top_cited_research_journal +=
            interest_score.append(float(Research_Interest_Score.replace(",", "").replace(".", "")))
            reads.append(float(Reads.replace(",", "").replace(".", "")))
            h_index.append(float(hindex.replace(",", "").replace(".", "")))
            URL.append(url)
            

https://www.researchgate.net/profile/Vladimir-Osipov-8
Vladimir Al. Osipov
Nan
Holon-Institute-of-Technology
801
Department of physics
Nonlinear Dynamics|Statistics|Probability
388.7
3,403
762
61
14
p-Adic Models of Ultrametric Diffusion Constrained by Hierarchical Energy Landscapes
PhD
38


https://www.researchgate.net/profile/Ronens-Sosnik
Ronens Sosnik
Nan
Holon-Institute-of-Technology
801
Faculty of Engineering
Brain Imaging|Cognitive Neuroscience|Neuroimaging
694.5
8,872
1,327
11
15
Transformation from temporal to rate coding in a somatosensory thalamocortical pathway
Senior Lecturer
43


https://www.researchgate.net/profile/Jacob-Gavan
Jacob. Gavan
Nan
Holon-Institute-of-Technology
801
Digital Signal Processing
Nan
99.0
2,775
351
0
10
Effects of desensitization on mobile radio system performance, part II: Quantitative analysis
Full Professor   Fellow IEEE
85


https://www.researchgate.net/profile/Y-Berkovich
Y. Berkovich
Nan
Holon-Institute-of-Technology
801
Faculty of Engineerin



https://www.researchgate.net/profile/Zeev-Ronen
Zeev Ronen
Nan
Ben-Gurion-University-of-the-Negev
401
Zuckerberg Institute for Water Research (ZIWR)
Microbiology|Soil|Hydraulics
2,296
40,597
4,105
101
36
Recycled vertical flow constructed wetland (RVFCW)-a novel method of recycling greywater for irrigation in small communities and households
PhD.
208


https://www.researchgate.net/profile/Mafatlal-Kher
Mafatlal M. Kher
Nan
Ariel-University
801
Department of Chemical Engineering and Biotechnology
Plant Tissue Culture|Somatic Embryogenesis|Micropropagation
651.8
52,916
438
273
12
Sandalwood: basic biology, tissue culture, and genetic transformation
PhD
44


https://www.researchgate.net/profile/Mona-Boaz-2
Mona Boaz
Nan
Ariel-University
801
Department of Nutrition Sciences
Statistical Modeling|Statistics|Hypertension
4,757
46,769
9,412
36
50
Secondary prevention with antioxidants of cardiovascular disease in endstage renal disease (SPACE): Randomised placebo-controlled trial
PhD, Epidem

In [124]:
df = pd.DataFrame({"Full name":fullname, "Gender":gender, "Position":position,
                   "Nation":nation, "Institute":institute, "Shanghai_ranking":shanghai_ranking,
                   "Depratment":depratment, "Disciplines":disciplines, "Publications":publications,
                   "Citations":citations, "Recommendations":recommendations,
                   "Top_cited_research":top_cited_research, "Interest_score":interest_score, 
                   "Reads":reads, "h_index":h_index, "URL":url})


df

df.to_csv('sientinst_data')